In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn import datasets
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import time
import matplotlib.pyplot as plt
# from sklearn_evaluation import plot

print("Imports complete")

Imports complete


In [2]:
# Load the data from the text file, ignoring the first line

filename="./data/MiniBooNE_PID.txt"
data=pd.read_csv(filename,delim_whitespace=True,skiprows=1,header=None)
print(data.head(1))
X=data

        0         1        2         3         4         5         6   \
0  2.59413  0.468803  20.6916  0.322648  0.009682  0.374393  0.803479   

         7        8         9   ...       40      41        42       43   44  \
0  0.896592  3.59665  0.249282  ...  101.174 -31.373  0.442259  5.86453  0.0   

         45        46        47        48        49  
0  0.090519  0.176909  0.457585  0.071769  0.245996  

[1 rows x 50 columns]


In [3]:
""" Create target vector for each row of the dataset. 
The first value in the first row contains the number of signal events, the second the number of background events. 
The target vector should contain 1 for signal events and 0 for background events."""

# Get number of signal and background events
signal, background = open(filename).readline().split()

y = np.concatenate((np.ones(int(signal)), np.zeros(int(background))))
print(y.shape)

(130064,)


In [4]:
# Standardize the input data
scaler = preprocessing.StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

In [5]:
""" save the scaled matrix and the target vector as npy files """
np.save('X.npy', X_scaled)
np.save('y.npy', y)

In [6]:
X=np.load('X.npy') # load the scaled matrix
y=np.load('y.npy')

In [7]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Create a vanilla neural network classifier and train it on the training data
nn_clf = MLPClassifier(random_state=42)

# Define the hyperparameter space to search over for the vanilla neural network classifier
nn_param_grid = {
    'hidden_layer_sizes': [(40,), (40,40), (40,40,40), (40,40,40,40)],
    # 'activation': ["relu", "logistic", "tanh", "identity"],
    # 'learning_rate_init': [0.001, 0.01, 0.1],
    'max_iter': [500],
    'random_state': [42]
}

# Set up the grid search using 5-fold cross-validation
nn_grid_search = GridSearchCV(nn_clf, nn_param_grid, cv=5, scoring='accuracy',verbose=10)
# nn_grid_search = RandomizedSearchCV(nn_clf, nn_param_grid, cv=5, scoring='accuracy', n_iter=2)

# Perform the grid search on the training set
nn_start_time = time.time()
nn_grid_search.fit(X_train, y_train)
nn_end_time = time.time()
print('Time required to grid search Vanilla Neural Network classifier:', nn_end_time - nn_start_time)

print('Vanilla Neural Network best hyperparameters:', nn_grid_search.best_params_)
print('Vanilla Neural Network best cross-validation accuracy:', nn_grid_search.best_score_)

In [ ]:
# Re-train the classifier using the best hyperparameters on the full training set
nn_clf = nn_grid_search.best_estimator_

nn_start_time = time.time()
nn_clf.fit(X_train, y_train)
nn_end_time = time.time()

# Evaluate the classifier on the test set
nn_test_accuracy = nn_grid_search.score(X_test, y_test)
print('Time required to train Vanilla Neural Network classifier:', nn_end_time - nn_start_time)
print('Vanilla Neural Network test accuracy:', nn_test_accuracy)

In [ ]:
# Get the results of the grid search
nn_results = nn_grid_search.cv_results_

nn_mean_scores = nn_results['mean_test_score']
nn_mean_times = nn_results['mean_fit_time']
nn_params = nn_results['param_hidden_layer_sizes']

# Loop through each combination of hyperparameters
for mean_score, mean_time, param in zip(nn_mean_scores, nn_mean_times, nn_params):
    print("HiddenLayerSize: %r | %.2fs | test-score: %f" % (param, mean_time, mean_score))

In [ ]:
""" plot the results of the grid search against the hyperparameter values """
fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.plot([1,2,3,4], nn_mean_scores, 'o:', label="Mean Score", linestyle=':')
ax2.plot([1,2,3,4], nn_mean_times, 'o:', color='orange', label="Mean Fit Time")
ax1.set_xlabel('Number of hidden layers of size 40')
ax1.set_ylabel('Score', color='blue')
ax2.set_ylabel('Fit time [s]', color='orange')

plt.show()

In [ ]:
from thundersvm import SVC

In [ ]:
# Create a SVM classifier and train it on the training data
svm_clf = SVC(kernel="rbf",random_state=42)

svm_param_grid = {
    'kernel': ['linear','rbf', 'poly', 'sigmoid'],
    'C': [10],
    'gamma': ['auto'],
    'random_state': [42],
}

svm_grid_search = GridSearchCV(svm_clf, svm_param_grid, cv=5, scoring='accuracy', verbose=10)

svm_start_time = time.time()
svm_grid_search.fit(X_train, y_train)
svm_end_time = time.time()

print('SVM best hyperparameters:', svm_grid_search.best_params_)
print('SVM best cross-validation accuracy:', svm_grid_search.best_score_)
print('Time required to perform GridSearch on SVM classifier:', svm_end_time - svm_start_time, 'seconds')

In [ ]:
svm_clf = svm_grid_search.best_estimator_

svm_clf.fit(X_train, y_train)

svm_test_accuracy = svm_grid_search.score(X_test, y_test)
print('Time required to train SVM classifier:', svm_end_time - svm_start_time)
print('SVM test accuracy:', svm_test_accuracy)

In [ ]:
# Get the results of the grid search
svm_results = svm_grid_search.cv_results_

svm_mean_scores = svm_results['mean_test_score']
svm_mean_score_times = svm_results['mean_score_time']
svm_mean_fit_times = svm_results['mean_fit_time']
svm_param_C = svm_results['param_C']

for mean_score, mean_score_time, mean_fit_time, param in zip(svm_mean_scores, svm_mean_score_times, svm_mean_fit_times, svm_param_C):
    print("C: %r | %.2fs | test-score: %f" % (param, mean_fit_time, mean_score))

In [ ]:
""" plot the results of the grid search against the hyperparameter values """
fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.plot(svm_param_C, svm_mean_scores,'o:', color='blue', label="Mean Score")
ax2.plot(svm_param_C, svm_mean_fit_times,'o:', color='orange', label="Mean Fit Time")
ax1.set_xlabel('Regularization parameter')
ax1.set_ylabel('Score', color='blue')
ax2.set_ylabel('Fit time', color='orange')

plt.show()

In [8]:
# Create a random forest classifier and train it on the training data
rf_clf = RandomForestClassifier(random_state=42)

rf_param_grid = {
    "n_estimators": [50, 100, 200],
    # "max_depth": [5, 10, 15],
    # "min_samples_split": [2, 4, 8],
    # "min_samples_leaf": [1, 2, 4],
    # "max_features": ["sqrt", "log2", None]
    'random_state': [42]
}

rf_grid_search = GridSearchCV(rf_clf, rf_param_grid, cv=5, scoring='accuracy', verbose=10)

rf_start_time = time.time()
rf_grid_search.fit(X_train, y_train)
rf_end_time = time.time()

print('RF best hyperparameters:', rf_grid_search.best_params_)
print('RF best cross-validation accuracy:', rf_grid_search.best_score_)
print('Time required to grid search for RF classifier:', rf_end_time - rf_start_time)

KeyboardInterrupt: 

In [ ]:
rf_clf = rf_grid_search.best_estimator_
rf_clf.fit(X_train, y_train)

rf_test_accuracy = rf_grid_search.score(X_test, y_test)
print('Time required to train RF classifier:', rf_end_time - rf_start_time)
print('RF test accuracy:', rf_test_accuracy)

In [ ]:
# Get the results of the grid search
rf_results = rf_grid_search.cv_results_
print(pd.DataFrame(rf_results))

rf_mean_scores = rf_results['mean_test_score']
rf_mean_fit_times = rf_results['mean_fit_time']

for mean_score, mean_score_time, mean_fit_time in zip(rf_mean_scores, rf_mean_score_times, rf_mean_fit_times):
    print("C: %r | %.2fs | test-score: %f" % (param, mean_fit_time, mean_score))

In [ ]:
""" plot the results of the grid search against the hyperparameter values """
fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.plot(svm_param_C, svm_mean_scores,'o:', color='blue', label="Mean Score")
ax2.plot(svm_param_C, svm_mean_fit_times,'o:', color='orange', label="Mean Fit Time")
ax1.set_xlabel('Regularization parameter')
ax1.set_ylabel('Score', color='blue')
ax2.set_ylabel('Fit time', color='orange')

plt.show()

In [ ]:
# Generate predictions for the test set
nn_predictions = nn_clf.predict(X_test)
svm_predictions = svm_clf.predict(X_test)
rf_predictions = rf_clf.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix

# Define a dictionary that maps model names to their hyperparameters
model_hyperparams = {
    'Vanilla Neural Network': nn_grid_search.best_params_,
    'SVM': svm_grid_search.best_params_,
    'Random Forest': rf_grid_search.best_params_
}

# Compute the confusion matrix for each model
nn_confusion_matrix = metrics.confusion_matrix(y_test, nn_predictions)
svm_confusion_matrix = metrics.confusion_matrix(y_test, svm_predictions)
rf_confusion_matrix = metrics.confusion_matrix(y_test, rf_predictions)

# Print the confusion matrix and hyperparameters for each model
print('Vanilla Neural Network confusion matrix:')
print(nn_confusion_matrix)
print('Vanilla Neural Network hyperparameters:', model_hyperparams['Vanilla Neural Network'])

print('SVM confusion matrix:')
print(svm_confusion_matrix)
print('SVM hyperparameters:', model_hyperparams['SVM'])

print('Random Forest confusion matrix:')
print(rf_confusion_matrix)
print('Random Forest hyperparameters:', model_hyperparams['Random Forest'])

In [ ]:
# Evaluate the performance of each classifier
nn_accuracy = metrics.accuracy_score(y_test, nn_predictions)
svm_accuracy = metrics.accuracy_score(y_test, svm_predictions)
rf_accuracy = metrics.accuracy_score(y_test, rf_predictions)
print("Vanilla Neural Network Accuracy:", nn_accuracy)
print("SVM Accuracy:", svm_accuracy)
print("Random Forest Accuracy:", rf_accuracy)

In [ ]:
from joblib import dump, load

In [ ]:
dump(nn_clf, 'nn_clf.joblib')

In [ ]:
dump(nn_grid_search, 'nn_grid_search.joblib')

In [ ]:
dump(svm_clf, 'svm_clf.joblib')

In [ ]:
dump(svm_grid_search, 'svm_grid_search.joblib')

In [ ]:
dump(rf_clf, 'rf_clf.joblib')

In [ ]:
dump(rf_grid_search, 'rf_grid_search.joblib')

In [ ]:
nn_clf = load('nn_clf.joblib')

In [ ]:
svm_clf = load('svm_clf.joblib')

In [ ]:
rf_clf = load('rf_clf.joblib')